In [1]:
import pandas as pd
from unidecode import unidecode

In [2]:
# DF with desired structure
target_df = pd.DataFrame(columns=['Categoria', 'Data', 'Descrição', 'Valor', 'Código Conta', 'Natureza', 'Recorrência'])
display(target_df)
# DF with initial values in each account
inicio_df = pd.DataFrame({'conta': ['NU', 'NP', 'CT', 'BB', 'RC'], 'saldo': [357.21, 0, 239.85, 120.25, 0]})
inicio_df

,Categoria,Data,Descrição,Valor,Código Conta,Natureza,Recorrência


,conta,saldo
0,NU,357.21
1,NP,0.00
2,CT,239.85
3,BB,120.25
4,RC,0.00


In [3]:
# Add initial values to target_df

for idx, (account, value) in inicio_df.iterrows():
    target_df.loc[len(target_df)] = ['Extra', pd.to_datetime('01/01/2020'), 'Saldo Inicial', value, account, 'Entrada', 'Única']
    
display(target_df.info())
target_df.head(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Categoria     5 non-null      object        
 1   Data          5 non-null      datetime64[ns]
 2   Descrição     5 non-null      object        
 3   Valor         5 non-null      float64       
 4   Código Conta  5 non-null      object        
 5   Natureza      5 non-null      object        
 6   Recorrência   5 non-null      object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 320.0+ bytes


None

,Categoria,Data,Descrição,Valor,Código Conta,Natureza,Recorrência
0,Extra,2020-01-01,Saldo Inicial,357.21,NU,Entrada,Única
1,Extra,2020-01-01,Saldo Inicial,0.00,NP,Entrada,Única
2,Extra,2020-01-01,Saldo Inicial,239.85,CT,Entrada,Única


In [4]:
def load_n_drop(year, idx):
    """Load excel files and drop empty rows

    Args:
        year (int): Year of the file to be load
        idx (int): Index of the sheet to be load

    Returns:
        DataFrame: A pandas dataframe with the desired sheet
    """
    data_raw = pd.read_excel(file_string+f"{year}.xlsx", sheet_name=sheets_names[idx], header=1, usecols=new_names_dict[idx][1])
    return data_raw.dropna(axis=0, how="all")

# Files path
file_string = f"Source_data/Controle de gastos - 20"
sheets_names = ["Entrada", "Saída", "Movimentação"]
new_names_dict = [("Entrada", "B:G"), ("Saida", "B:G"), ("Movimentacao", "B:I")]

# Each element of the list is a dictionary, each dictionary represents a loaded file with your sheets
sheet_list = []
# Read all files
for year in range(20, 23):
    sheet_list.append({element[0]: load_n_drop(year, idx) for idx, element in enumerate(new_names_dict)})

sheet_list[0].keys()

dict_keys(['Entrada', 'Saida', 'Movimentacao'])

In [5]:
# Verify absence of values in columns, if there is any NaN value, print a message
# showing where the NaN value is
for year, sheet in zip(range(2020, 2023), sheet_list):
    for key in sheet.keys():
        result = sheet[key].isna().sum()
        if result.sum() > 0:
            print(f"{year} - {key} has NaN values\n{result}")

def load_in_out(tp, nat):
    # Add value 'Entrada' and 'Saída' to target_df
    aux = sheet_list[0][tp][['Categoria', 'Dia', 'Descrição', 'Valor', 'Código']]
    for sheet_dict in sheet_list[1:]:
        aux = pd.concat([aux, sheet_dict[tp][['Categoria', 'Dia', 'Descrição', 'Valor', 'Código']]])
        
        
    aux['Natureza'] = nat
    aux['Recorrência'] = 'Única' # Adicionar outros tipos de recorrência ex. Mensal e semanal ( baseado no nome da descrição)
    
    aux.rename(columns={'Dia': 'Data', 'Código': 'Código Conta'}, inplace=True)
        
    return aux
    

target_df = pd.concat([target_df, load_in_out("Entrada", "Entrada")], ignore_index=True)
target_df = pd.concat([target_df, load_in_out("Saida", "Saída")], ignore_index=True)

In [6]:

def load_mov(tp, nat):
    # Transform "Movimentação" in "Entrada" and "Saída" Operations
    aux = sheet_list[0]["Movimentacao"][['Categoria', 'Dia', 'Descrição', 'Valor', tp]]
    for sheet_dict in sheet_list[1:]:
        aux = pd.concat([aux, sheet_dict["Movimentacao"][['Categoria', 'Dia', 'Descrição', 'Valor', tp]]])
        
        
    aux['Natureza'] = nat
    aux['Recorrência'] = 'Única' # Adicionar outros tipos de recorrência ex. Mensal e semanal ( baseado no nome da descrição)
    
    aux.rename(columns={'Dia': 'Data', tp: 'Código Conta'}, inplace=True)
        
    return aux

# Transform "Movimentação" in "Entrada" and "Saída" Operations
target_df = pd.concat([target_df, load_mov("Cód. Destino", "Entrada")], ignore_index=True)
target_df = pd.concat([target_df, load_mov("Cód. Origem", "Saída")], ignore_index=True)

# Check Data

In [7]:
# Check data
mask = (target_df.Data.dt.year <= 2022)
aux = pd.crosstab(target_df.loc[mask, "Código Conta"], target_df.loc[mask, "Natureza"], target_df.loc[mask, "Valor"], aggfunc="sum").fillna(0)
aux["Diff"] = aux["Entrada"] - aux["Saída"]
# aux["check"] = ["X", "OK", "OK", "X", "X", "OK", "OK", "OK", "OK"]
aux

Natureza,Entrada,Saída,Diff
Código Conta,,,
BB,47531.37,47531.37,0.00
BT,1050.00,0.00,1050.00
CT,13041.30,12939.30,102.00
NP,46156.93,32864.49,13292.44
NU,119316.96,118921.47,395.49
NUCXSM,558.01,558.01,0.00
PP,500.61,500.61,0.00
PS,11925.73,1609.80,10315.93
RC,12606.84,0.00,12606.84


In [8]:
# Replace Código Conta with account names
accounts = ["Banco do Brasil Corrente", "Binance Cripto", "Carteira", "Nubank Poupança", \
    "Nubank Corrente", "Nubank Caixa São Miguel", "PicPay Investimento", "PagSeguro Investimento",\
        "Rico Investimento"]

accounts_dict = {key: value for key, value in zip(sorted(target_df["Código Conta"].unique()), accounts)}

# Replace values in Código Conta by values in dict accounts_dict
target_df["Código Conta"].replace(accounts_dict, inplace=True)
target_df.head(3)

,Categoria,Data,Descrição,Valor,Código Conta,Natureza,Recorrência
0,Extra,2020-01-01,Saldo Inicial,357.21,Nubank Corrente,Entrada,Única
1,Extra,2020-01-01,Saldo Inicial,0.00,Nubank Poupança,Entrada,Única
2,Extra,2020-01-01,Saldo Inicial,239.85,Carteira,Entrada,Única


In [9]:
target_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1797 entries, 0 to 1796
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Categoria     1797 non-null   object        
 1   Data          1797 non-null   datetime64[ns]
 2   Descrição     1797 non-null   object        
 3   Valor         1797 non-null   float64       
 4   Código Conta  1797 non-null   object        
 5   Natureza      1797 non-null   object        
 6   Recorrência   1797 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 98.4+ KB


# Credit Cards

In [10]:
# DF with desired structure
cols = ['Categoria', 'Data', 'Descrição', 'Valor', 'Nº Parcelas', 'Cartão', 'Recorrência']
target_cc_df = pd.DataFrame(columns=cols)
display(target_cc_df)
# DF with initial values in each account
inicio_cc_df = pd.DataFrame({'codigo': ['NUCC'], 'saldo': [-1619.18]})
inicio_cc_df

,Categoria,Data,Descrição,Valor,Nº Parcelas,Cartão,Recorrência


,codigo,saldo
0,NUCC,-1619.18


In [11]:
# Files path
file_string = f"Source_data/Controle de gastos - 20"
limits = {20: "L:Q", 21: "L:R", 22: "L:R"}

# Each element of the list is a dictionary, each dictionary represents a loaded file with your sheets
sheet_cc_dict = {}
# Read all files
for year in range(20, 23):
    sheet_cc_dict[year] = pd.read_excel(f"{file_string}{year}.xlsx", sheet_name="Saída", header=1, usecols=limits[year]).dropna(axis=0, how="all")
    
# Fix 2020 sheet
sheet_cc_dict[20]["Categoria"] = "Extra"
sheet_cc_dict[20]["Recorrência"] = "Única"
sheet_cc_dict[20].rename(columns={"Dia.1": "Data", "Descrição.1": "Descrição", "Valor total": "Valor",\
                                    "Parcelas": "Nº Parcelas"}, inplace=True)
sheet_cc_dict[20] = sheet_cc_dict[20][cols]

for idx in [21, 22]:
    sheet_cc_dict[idx]["Recorrência"] = "Única"
    sheet_cc_dict[idx].rename(columns={"Categoria.1": "Categoria", "Dia.1": "Data", "Descrição.1": "Descrição", "Valor total": "Valor",\
                                        "Parcelas": "Nº Parcelas"}, inplace=True)
    sheet_cc_dict[idx] = sheet_cc_dict[21][cols]

In [12]:
# Fix "categoria" in 2020 sheet
fix_cat_dict = {"uber": "Transporte", "farmacia": "Saúde", "aliexpr": "Produtos", "99": "Transporte",\
                "alura": "Educação", "inside": "Educação", "bangood": "Produtos", "duval": "Extra",\
                    "vitao": "Lazer", "invasao": "Lazer", "google": "Produtos", "receptor": "Produtos",\
                        "correios": "Serviços", "frete": "Serviços", "kindle": "Produtos", "agae": "Produtos",\
                            "peca": "Produtos", "genobio": "Educação", "wix": "Serviços"}

for idx, row in sheet_cc_dict[20].iterrows():
    desc = unidecode(str(row["Descrição"])).lower()
    for key in fix_cat_dict.keys():
        if key in desc:
            sheet_cc_dict[20].loc[idx, "Categoria"] = fix_cat_dict[key]

# Fix descrição in 20202 sheet            
mask = sheet_cc_dict[20]["Descrição"].str.contains("Uber").fillna(False)
sheet_cc_dict[20].loc[mask, "Descrição"] = "Uber"

mask = sheet_cc_dict[20]["Descrição"].str.contains("99").fillna(True)
sheet_cc_dict[20].loc[mask, "Descrição"] = "99 Pop"


In [13]:
cc_df = pd.concat([sheet_cc_dict[20], sheet_cc_dict[21], sheet_cc_dict[22]], ignore_index=True)
cc_df.head(3)

,Categoria,Data,Descrição,Valor,Nº Parcelas,Cartão,Recorrência
0,Transporte,2020-01-09,Uber,8.83,1.0,Nubank,Única
1,Saúde,2020-01-13,Farmácia,24.42,1.0,Nubank,Única
2,Transporte,2020-01-17,Uber,10.22,1.0,Nubank,Única


# Save data

In [14]:
# Save target_df and cc_df
target_df.to_csv("Ready_to_load/entrada_saida_2020_2022.csv", index=False)
cc_df.to_csv("Ready_to_load/cartoes_2020_2022.csv", index=False)